In [4]:
# !pip install langchain langchain-core langchain-huggingface

In [5]:
# !pip install streamlit requests beautifulsoup4

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [30]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
import requests
from bs4 import BeautifulSoup
import random  # For simulation randomness

# Step 1: Define the Custom Tools

In [46]:
class HistoricalQuery(BaseModel):
    query: str = Field(description="The historical fact to query, e.g., 'Fall of Roman Empire date'")

@tool(args_schema=HistoricalQuery)
def historical_fact_checker(query: str) -> str:
    """Fetches verified historical facts from Wikipedia. Use this to ground speculations in real events."""
    
    # Proper title formatting (underscores for API)
    title = query.strip().replace(" ", "_")
    
    # Correct parameters: exintro=1 (or True), explaintext=1
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": True,        # Only the intro section
        "explaintext": True,    # Plain text, no HTML
        "redirects": True,      # Follow redirects automatically
        "titles": title
    }
    
    headers = {
        "User-Agent": "YourAppName/1.0 (your.email@example.com)"  # Replace with your info
    }
    
    try:
        response = requests.get("https://en.wikipedia.org/w/api.php", params=params, headers=headers, timeout=10)
        response.raise_for_status()  # Raises HTTPError for 4xx/5xx
        
        data = response.json()
        
        pages = data["query"]["pages"]
        page = next(iter(pages.values()))  # Get the first (and usually only) page
        
        if "missing" in page or page.get("extract") in (None, ""):
            return f"No Wikipedia page found for '{query}'. Try a different phrasing or more precise title."
        
        extract = page["extract"]
        # return extract[:1000] + ("..." if len(extract) > 1000 else "")  # Truncate reasonably
        return extract

    except requests.exceptions.RequestException as e:
        return f"Error fetching from Wikipedia: {str(e)}"
    except (KeyError, ValueError) as e:
        return f"Error parsing Wikipedia response: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [47]:
historical_fact_checker.invoke({'query':'Mahabharata'})


'The Mahābhārata ( mə-HAH-BAR-ə-tə, MAH-hə-; Sanskrit: महाभारतम्, IAST: Mahābhāratam, pronounced [mɐɦaːˈbʱaːrɐt̪ɐm]) is a smriti text (also described as a Sanskrit epic) from ancient India, one of the two important epics of Hinduism known as the Itihasas, the other being the Ramayana. It narrates the events and aftermath of the Kurukshetra War, a war of succession between two groups of princely cousins, the Kauravas and the Pāṇḍavas. It  contains philosophical and devotional material, such as a discussion of the four "goals of life" or puruṣārtha (12.161). Among the principal works and stories in the Mahābhārata are the Bhagavad Gita, the story of Damayanti, Shakuntala, Pururava and Urvashi, Savitri and Satyavan, Kacha and Devayani, Rishyasringa and an abbreviated version of the Rāmāyaṇa.\n\nTraditionally, the authorship of the Mahābhārata is attributed to Vyāsa. There have been many attempts to unravel its historical growth and compositional layers. The bulk of the Mahābhārata was pro

In [48]:
class SimulationInput(BaseModel):
    original_event: str = Field(description="The real historical event to alter")
    alteration: str = Field(description="The 'what-if' change")
    years_ahead: int = Field(description="Number of years to simulate forward", default=100)

@tool(args_schema=SimulationInput)
def timeline_simulator(original_event: str, alteration: str, years_ahead: int = 100) -> str:
    """Simulates an alternate timeline by forking history. Outputs a list of key hypothetical events."""
    # Simple rule-based simulation (expandable with more logic)
    impacts = [
        f"{original_event} is altered to {alteration}.",
        f"{random.randint(1, 10)} years later: Technological boom in related fields.",
        f"{random.randint(10, 50)} years later: Geopolitical shifts, e.g., new alliances.",
        f"{years_ahead} years later: Modern world equivalent with {alteration}-influenced culture."
    ]
    return "\n".join(impacts)

In [51]:
timeline_simulator.invoke({"original_event": "Mahabharat", "alteration": "Future Culture", "years_ahead": 100})

'Mahabharat is altered to Future Culture.\n2 years later: Technological boom in related fields.\n12 years later: Geopolitical shifts, e.g., new alliances.\n100 years later: Modern world equivalent with Future Culture-influenced culture.'

In [52]:
class ImpactInput(BaseModel):
    scenario: str = Field(description="The alternate scenario description")
    factors: list[str] = Field(description="List of factors to analyze, e.g., ['economy', 'technology']")

@tool(args_schema=ImpactInput)
def impact_analyzer(scenario: str, factors: list[str]) -> dict:
    """Analyzes hypothetical impacts on society, economy, etc. Returns a dict of scores (0-10)."""
    # Dummy quantitative model; in reality, use APIs or ML for depth
    results = {}
    for factor in factors:
        score = random.randint(3, 10)  # Simulate calculation
        results[factor] = f"Impact on {factor}: {score}/10 - {scenario} would accelerate {factor}."
    return results

In [53]:
impact_analyzer.invoke({"scenario": "", "factors": ["factor_1", "factor_2"]})

{'factor_1': 'Impact on factor_1: 4/10 -  would accelerate factor_1.',
 'factor_2': 'Impact on factor_2: 9/10 -  would accelerate factor_2.'}

In [54]:
class NarrativeInput(BaseModel):
    facts: str = Field(description="Historical facts")
    simulation: str = Field(description="Simulated timeline")
    impacts: dict = Field(description="Impact analysis results")

@tool(args_schema=NarrativeInput)
def narrative_generator(facts: str, simulation: str, impacts: dict) -> str:
    """Generates a cohesive narrative story from facts, simulation, and impacts. Use last in chain."""
    # Simple concatenation; enhance with prompts if needed
    story = f"Based on facts: {facts}\nIn this alternate timeline: {simulation}\nImpacts: {str(impacts)}\n\nFull Story: Once upon a time..."
    # Expand with creative logic here
    return story

In [55]:
narrative_generator.invoke({"facts": "", "simulation": "", "impacts": {"impacts_1": "impacts_1"} })

"Based on facts: \nIn this alternate timeline: \nImpacts: {'impacts_1': 'impacts_1'}\n\nFull Story: Once upon a time..."

# Step 2: Build the Agent

In [71]:

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

tools = [historical_fact_checker, timeline_simulator, impact_analyzer, narrative_generator]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Alternate History Explorer. Use tools step-by-step: First check facts, then simulate, analyze impacts, and finally generate narrative. Avoid speculation without tools. Be ethical and fun."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),  # For intermediate steps
])

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)  # Verbose for debugging

In [72]:
result = agent_executor.invoke({"input": "What if World War Two never happened"})
print(result['output'])

### Step 1: Check Facts

In reality, World War II was a pivotal event in human history, involving the majority of the world's nations and producing a series of significant changes in global politics, economics, and technology.

### Step 2: Simulate the Alteration

If World War II had never happened, several key historical events and developments would be altered. Here’s a step-by-step simulation of this alternate timeline:

1. **1939 - Technological Boom in Related Fields:**
   - Without the need to defend against an aggressive Axis power, countries like the United States and the Soviet Union might have focused more resources on scientific and technological advancements unrelated to war.
   - This could lead to faster development in fields such as electronics, computing, and medicine.

2. **1950 - Geopolitical Shifts:**
   - The absence of World War II could have led to different alliances and political dynamics.
   - For example, the United States might have made different choices in 